# Data Cleaning Listado de Listado de Beneficiarios 2023

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2023 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios 2023 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario ->
4. Dataset_Inegi ->

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

/opt/anaconda3/lib/python3.11/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

### Lectura del listado de Beneficiarios 2023

In [4]:
listado_beneficiarios, sumOfRows = load_datasets("../../data/beneficiarios2023")

Number of rows in DataFrame 1: 42854
Number of rows in DataFrame 2: 493598
Number of rows in DataFrame 3: 830761
Number of rows in DataFrame 4: 312892


In [5]:
sumOfRows

1680105

In [6]:
listado_beneficiarios.head()

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0


# 2. Limpieza de los datos

### 2.1 INEGI

In [7]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [8]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [9]:
# Renombrar las columnas
dataset_inegi.rename(columns={
    'CVE_ENT': 'CVE_ENT',
    'NOM_ENT': 'Entidad_inegi',
    'CVE_MUN': 'CVE_MUN',
    'NOM_MUN': 'Municipio_inegi'
}, inplace=True)


# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT            object
Entidad_inegi      object
CVE_MUN            object
Municipio_inegi    object
dtype: object

In [11]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_uniquemun = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_uniquemun: ", dataset_inegi_uniquemun.shape)

Shape of dataset_inegi:  (299568, 4)
Shape of dataset_inegi_uniquemun:  (2476, 4)


A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [12]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_uniquemun.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,01,Aguascalientes,001,Aguascalientes
708,01,Aguascalientes,002,Asientos
945,01,Aguascalientes,003,Calvillo
1237,01,Aguascalientes,004,Cosío
1330,01,Aguascalientes,005,Jesús María


In [13]:
dataset_inegi_uniquemun.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2476 entries, 0 to 299526
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CVE_ENT          2476 non-null   object
 1   Entidad_inegi    2476 non-null   object
 2   CVE_MUN          2476 non-null   object
 3   Municipio_inegi  2476 non-null   object
dtypes: object(4)
memory usage: 96.7+ KB


In [14]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_uniquemun.nunique())

Los valores únicos en cada columna son:
 CVE_ENT              32
Entidad_inegi        32
CVE_MUN             570
Municipio_inegi    2332
dtype: int64


#### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [15]:
# Estandarizamos la limpieza de los datos
dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)

/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/4159994235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/4159994235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)


In [16]:
# Creamos una columna con la clave única por municipio

dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[
    'Municipio_c_inegi'].astype(str)

dataset_inegi_uniquemun.head()

/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/2100087808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[


,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
0,01,Aguascalientes,001,Aguascalientes,aguascalientes,aguascalientes,aguascalientes-aguascalientes
708,01,Aguascalientes,002,Asientos,aguascalientes,asientos,aguascalientes-asientos
945,01,Aguascalientes,003,Calvillo,aguascalientes,calvillo,aguascalientes-calvillo
1237,01,Aguascalientes,004,Cosío,aguascalientes,cosio,aguascalientes-cosio
1330,01,Aguascalientes,005,Jesús María,aguascalientes,jesus maria,aguascalientes-jesus maria


In [17]:
dataset_inegi_uniquemun.tail()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
299150,32,Zacatecas,054,Villa Hidalgo,zacatecas,villa hidalgo,zacatecas-villa hidalgo
299211,32,Zacatecas,055,Villanueva,zacatecas,villanueva,zacatecas-villanueva
299363,32,Zacatecas,056,Zacatecas,zacatecas,zacatecas,zacatecas-zacatecas
299484,32,Zacatecas,057,Trancoso,zacatecas,trancoso,zacatecas-trancoso
299526,32,Zacatecas,058,Santa María de la Paz,zacatecas,santa maria de la paz,zacatecas-santa maria de la paz


## 2.2 Beneficiarios 2023

### Creación de Estados_Beneficiarios2023
Este dataset es una versión de Beneficiarios2023 pero más ligera y sin repeticiones.

In [19]:
listado_beneficiarios.dtypes

ESTADO               object
MUNICIPIO            object
ACUSE ESTATAL        object
APELLIDO PATERNO     object
APELLIDO MATERNO     object
NOMBRE (S)           object
PAQUETE             float64
dtype: object

In [20]:
descriptive_stats = listado_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                      count   unique                             top    freq  \
ESTADO              1680105       30                        GUERRERO  331097   
MUNICIPIO           1680105     2237                  LAS MARGARITAS   18041   
ACUSE ESTATAL       1680105  1680105  23-PROESFE-ESTR-000051-E000-SL       1   
APELLIDO PATERNO    1680104    14995                       HERNANDEZ   77418   
APELLIDO MATERNO    1653877    17604                       HERNANDEZ   76953   
NOMBRE (S)          1680105   124628                            JUAN   26798   
PAQUETE           1680097.0      NaN                             NaN     NaN   

                      mean       std  min  25%  50%  75%   max  
ESTADO                 NaN       NaN  NaN  NaN  NaN  NaN   NaN  
MUNICIPIO              NaN       NaN  NaN  NaN  NaN  NaN   NaN  
ACUSE ESTATAL          NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO PATERNO       NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO MATERNO       NaN       N

In [23]:
listado_beneficiarios.isna().sum()

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26228
NOMBRE (S)              0
PAQUETE                 8
dtype: int64

In [25]:
listado_beneficiarios.head()

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0


In [27]:
Estados_beneficiarios2023 = listado_beneficiarios[['ESTADO', 'MUNICIPIO']]

In [28]:
# Estandarizamos la limpieza de los datos
Estados_beneficiarios2023['ESTADO_c_benef'] = Estados_beneficiarios2023['ESTADO'].apply(clean_text)
Estados_beneficiarios2023['MUNICIPIO_c_benef'] = Estados_beneficiarios2023['MUNICIPIO'].apply(clean_text)

/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/4050903399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios2023['ESTADO_c_benef'] = Estados_beneficiarios2023['ESTADO'].apply(clean_text)
/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/4050903399.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios2023['MUNICIPIO_c_benef'] = Estados_beneficiarios2023['MUNICIPIO'].apply(clean_text)


In [29]:
Estados_beneficiarios2023.head()

,ESTADO,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef
0,SINALOA,AHOME,sinaloa,ahome
1,SINALOA,AHOME,sinaloa,ahome
2,SINALOA,AHOME,sinaloa,ahome
3,SINALOA,AHOME,sinaloa,ahome
4,SINALOA,AHOME,sinaloa,ahome


In [31]:
Estados_beneficiarios2023['KEY_benef'] = Estados_beneficiarios2023['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2023[
    'MUNICIPIO_c_benef'].astype(str)

Estados_beneficiarios2023.head()

/var/folders/lr/s5n5xs912wb6j0x5kb_y8br00000gn/T/ipykernel_29731/4066468156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios2023['KEY_benef'] = Estados_beneficiarios2023['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2023[


,ESTADO,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef,KEY_benef
0,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
1,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
2,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
3,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
4,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome


In [ ]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Estados_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2023

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2023. Para ello haremos un Left join entre Estados_Beneficiarios2023 y dataset_inegi_clean.

### 3.1 Creamos las columnas clave

Crearemos las columnas clave concatenando [Estado_limpio]-[Municipio_limpio] en **ambos datasets**

In [ ]:
# INEGI
dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY_inegi"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

In [ ]:
Estados_beneficiarios["ESTADO_Clean"] = Estados_beneficiarios["ESTADO_Clean"].astype(str)
Estados_beneficiarios["MUNICIPIO_Clean"] = Estados_beneficiarios["MUNICIPIO_Clean"].astype(str)

Estados_beneficiarios["KEY_benef23"] = Estados_beneficiarios["ESTADO_Clean"] + "-" + Estados_beneficiarios["MUNICIPIO_Clean"]

In [ ]:
df = Estados_beneficiarios['KEY_benef23'].unique()

In [ ]:
df.shape

### 3.1 Left join

Creamos el diccionario.

In [ ]:
Estados_beneficiarios = Estados_beneficiarios.drop(['ESTADO', 'MUNICIPIO'], axis=1)
Estados_beneficiarios = Estados_beneficiarios.drop_duplicates()
Estados_beneficiarios.shape

In [ ]:
Estados_beneficiarios

In [ ]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge_beneficiarios2023(df_inegi, df_prod, key1, key2, threshold=90, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='inner',
                         suffixes=('_inegi', '_prod'))
    
    return df_merged


In [ ]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge_beneficiarios2023(dataset_inegi_clean, Estados_beneficiarios, 'KEY_inegi', 'KEY_benef23')
diccionario.drop_duplicates(subset=['KEY_inegi'], inplace=True)

# Mostrar el resultado
diccionario.columns

In [ ]:
nan_rows = diccionario.isna()

In [ ]:
diccionario.to_csv('../../data/diccionario.csv', index=False)

## 3.2 Listado beneficiarios2023

Esta sección se encarga de completar el listado original de Beneficiarios Autorizados con los nombre corregido de INEGI usando el diccionario.

In [ ]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_beneficiarios['ESTADO_Clean'] = listado_beneficiarios['ESTADO'].apply(clean_text)
listado_beneficiarios['MUNICIPIO_Clean'] = listado_beneficiarios['MUNICIPIO'].apply(clean_text)
listado_beneficiarios['Estado-mun-KEY'] = listado_beneficiarios['ESTADO_Clean'].astype(str) + '-' + listado_beneficiarios[
    'MUNICIPIO_Clean'].astype(str)

In [ ]:
nan_rows = listado_beneficiarios.isna()

In [ ]:
nan_rows.sum()

In [ ]:
listado_beneficiarios.shape

In [ ]:
diccionario.shape

In [ ]:
# Lectura del diccionario manipulado
diccionario_manipulado = pd.read_csv('../../data/Diccionario_manual.csv')

In [ ]:
# Hacer el join
listado_beneficiarios_complete = pd.merge(listado_beneficiarios, diccionario, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef23", how='left', suffixes=('_prod', '_inegi'))

In [ ]:
listado_beneficiarios_complete[['CVE_ENT', 'CVE_MUN']] = listado_beneficiarios_complete['CVE_MUN_Unique'].str.split('-',
                                                                                                                expand=True)


In [ ]:
listado_beneficiarios_complete.columns

In [ ]:
# Seleccionamos las columnas que nos interesan
listado_beneficiarios_complete = listado_beneficiarios_complete[
    ['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_benef23', 'NOM_ENT', 'NOM_MUN', 'CVE_ENT', 'CVE_MUN']]

In [ ]:
# Revisamos el dataset
print(listado_beneficiarios_complete.shape)
print(listado_beneficiarios_complete.columns)
print(listado_beneficiarios_complete.head())

In [ ]:
nan_counts = listado_beneficiarios_complete.isna().sum()
print(nan_counts)

In [ ]:
nan_rows = listado_beneficiarios_complete[listado_beneficiarios_complete['NOM_ENT'].isna()]


In [ ]:
nan_rows.tail(50)

In [ ]:
listado_beneficiarios_complete = listado_beneficiarios_complete.dropna(subset=['PAQUETE'])

In [ ]:
listado_beneficiarios_complete.shape

In [ ]:
listado_beneficiarios_complete = listado_beneficiarios_complete.astype({
    'ESTADO': 'str',
    'MUNICIPIO': 'str',
    'ACUSE ESTATAL': 'str',
    'APELLIDO PATERNO': 'str',
    'APELLIDO MATERNO': 'str',
    'NOMBRE (S)': 'str',
    'PAQUETE': 'int',
    'NOM_MUN': 'str',
    'NOM_ENT': 'str',
    'CVE_MUN': 'str',
    'CVE_ENT': 'str',
    'KEY_benef23': 'str'

    })

listado_beneficiarios_complete = listado_beneficiarios_complete.rename(columns={
'ESTADO': 'estado1',
'MUNICIPIO': 'municipio1',
'ACUSE ESTATAL': 'acuse',
'APELLIDO PATERNO': 'apellido_paterno',
'APELLIDO MATERNO': 'apellido_materno',
'NOMBRE (S)': 'nombre_propio',
'PAQUETE': 'paquete',
'NOM_MUN': 'municipio',
'NOM_ENT': 'entidad',
'CVE_MUN': 'cve_mun',
'CVE_ENT': 'cve_ent',
'KEY_benef23': 'key_benef23'
})

In [ ]:
listado_beneficiarios_complete = listado_beneficiarios_complete.drop(columns=['estado1', 'municipio1'])

listado_beneficiarios_complete['id'] = listado_beneficiarios_complete.index

# Assuming df is your DataFrame
ordered_columns = ['id', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'acuse', 'apellido_paterno', 'apellido_materno', 'nombre_propio', 'paquete', 'key_benef23']
listado_beneficiarios_complete = listado_beneficiarios_complete.reindex(columns=ordered_columns)

listado_beneficiarios_complete['cve_ent'] = listado_beneficiarios_complete['cve_ent'].str.zfill(2)
listado_beneficiarios_complete['cve_mun'] = listado_beneficiarios_complete['cve_mun'].str.zfill(3) 

In [ ]:
listado_beneficiarios_complete.to_csv('../../data/listado_beneficiarios_complete.csv', index=False)